In [1]:
import graphlab as gl

In [2]:
def clean_data(path):
    import graphlab as gl
    sf = gl.SFrame.read_csv(path, column_type_hints={"rating":int})
    sf = sf.sample(0.001)
    return sf

def train_model(data):
    import graphlab as gl
    model = gl.recommender.create(data, target='rating', verbose=False)
    return model

def gen_recs(model, data):
    recs = model.recommend(data['user_id'])
    return recs

def my_batch_job(path):
    data = clean_data(path)
    model = train_model(data)
    recs = gen_recs(model, data)
    return recs

job = gl.deploy.job.create(my_batch_job, path="data/user_restaurant_rating.csv")

[INFO] graphlab.deploy.job: Validating job.
INFO:graphlab.deploy.job:Validating job.
[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1466882267.log
INFO:graphlab.cython.cy_server:GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1466882267.log
[INFO] graphlab.deploy.job: Creating a LocalAsync environment called 'async'.
INFO:graphlab.deploy.job:Creating a LocalAsync environment called 'async'.
[INFO] graphlab.deploy.job: Validation complete. Job: 'my_batch_job-Jun-25-2016-12-17-47' ready for execution.
INFO:graphlab.deploy.job:Validation complete. Job: 'my_batch_job-Jun-25-2016-12-17-47' ready for execution.
[INFO] graphlab.deploy.job: Job: 'my_batch_job-Jun-25-2016-12-17-47' scheduled.
INFO:graphlab.deploy.job:Job: 'my_batch_job-Jun-25-2016-12-17-47' scheduled.


This non-commercial license of GraphLab Create is assigned to dconger2@gmail.com and will expire on June 21, 2017. For commercial licensing options, visit https://dato.com/buy/.


In [3]:
job.get_status()

u'Completed'

In [4]:
recs = job.get_results()

In [5]:
recs

user_id,item_id,score,rank
tvXM9NGfiQIEAnvkOSQo1w,U2dZYCgkuS9SGgyKaCUlbQ,3.5911135989,1
tvXM9NGfiQIEAnvkOSQo1w,3AlN2XZ0I0F8650y2yjLWA,3.58434148167,2
tvXM9NGfiQIEAnvkOSQo1w,l3L25Yg_8E11tsS2D8ml-g,3.5401365516,3
tvXM9NGfiQIEAnvkOSQo1w,sIyHTizqAiGu12XMLX3N3g,3.51550889213,4
tvXM9NGfiQIEAnvkOSQo1w,j_pce4pG9krrBeYwUni8Pg,3.51211473543,5
tvXM9NGfiQIEAnvkOSQo1w,7h0-KtLbYH1gu6QBv_-jxg,3.51066900242,6
tvXM9NGfiQIEAnvkOSQo1w,itFwrNtjrQL5NPMYTeD44g,3.51005444128,7
tvXM9NGfiQIEAnvkOSQo1w,jlLCgHVC3gzCkfnlxuUZHA,3.51000080455,8
tvXM9NGfiQIEAnvkOSQo1w,OHsuvX4rmhnPvvRBb-QK7A,3.50929973472,9
tvXM9NGfiQIEAnvkOSQo1w,lliksv-tglfUz1T3B3vgvA,3.50911319453,10


In [9]:
@gl.deploy.required_packages(['mysql-connector-python'])
def persist_to_db(recs, dbhost, dbuser, dbpass, dbport, dbtable, dbname):
    import mysql.connector
    from mysql.connector import errorcode
    
    conn = mysql.connector.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport)
    conn.database = dbname
    cursor = conn.cursor()
    # this example expects the table to be empty, minor changes here if you want to 
    # update existing users' recommendations instead.
    add_row_sql = ("INSERT INTO " + dbtable + " (user_id, item_id, score, rank) "
                   "VALUES (%(user_id)s, %(item_id)s, %(score)s, %(rank)s)")

    print "Begin - Writing recommendations to DB...."
    for row in recs:
        cursor.execute(add_row_sql, row)
    print "End - Writing recommendations to DB...."
    
    # commit recommendations to database
    conn.commit()

In [10]:
job = gl.deploy.job.create(persist_to_db, 
                 recs = recs,
                 dbhost = '127.0.0.1', # change these db params appropriately
                 dbuser = 'root',
                 dbpass = '',
                 dbname = 'thesis_practice',
                 dbport = 3306,
                 dbtable = 'recommendations') 

results = job.get_results()

[INFO] graphlab.deploy.job: Validating job.
INFO:graphlab.deploy.job:Validating job.
[INFO] graphlab.deploy.job: Validation complete. Job: 'persist_to_db-Jun-25-2016-12-28-54' ready for execution.
INFO:graphlab.deploy.job:Validation complete. Job: 'persist_to_db-Jun-25-2016-12-28-54' ready for execution.
[INFO] graphlab.deploy.job: Job: 'persist_to_db-Jun-25-2016-12-28-54' scheduled.
INFO:graphlab.deploy.job:Job: 'persist_to_db-Jun-25-2016-12-28-54' scheduled.
[INFO] graphlab.deploy._job: Waiting for job to finish, this may take quite a while.
INFO:graphlab.deploy._job:Waiting for job to finish, this may take quite a while.
[INFO] graphlab.deploy._job: You may CTRL-C to stop this command and it will not cancel your job.
INFO:graphlab.deploy._job:You may CTRL-C to stop this command and it will not cancel your job.


In [12]:
job.get_status()

u'Completed'

In [13]:
results